In [1]:
import os
import glob
import pandas as pd
import calendar
import math
import itertools
import datetime
import win32com.client as win32
#from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [300]:
#Scrapping dates

url = "https://www.ecb.europa.eu/pub/projections/html/all-releases.en.html"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

#Creating dataframe
dat = soup.find_all("dt")
dats=[0 for i in range((len(dat)))]
for i in range(0,len(dat)): 
    dats[i]=dat[i]["isodate"]

#removing duplicates, noisy data and reseting index
dates = pd.DataFrame(dats, columns=["Dates"]).drop_duplicates()
dates = dates.drop(labels=[6,11,16,21,26,31,36,38,41,46,51,56,61,64,67], axis=0).reset_index()
dates = dates.drop('index', axis=1)

In [301]:
#Completing with missing dates
dates.loc[78, "Dates"] = '2004-03-04'
dates.loc[79, "Dates"] = '2003-12-04'
dates.loc[80, "Dates"] = '2003-09-04'
dates.loc[81, "Dates"] = '2003-06-05'
dates.loc[82, "Dates"] = '2003-03-06'
dates.loc[83, "Dates"] = '2002-12-05'
dates.loc[84, "Dates"] = '2002-09-05'
dates.loc[85, "Dates"] = '2002-06-06'
dates.loc[86, "Dates"] = '2002-03-07'
dates.loc[87, "Dates"] = '2001-12-06'
dates.loc[88, "Dates"] = '2001-09-06'
dates.loc[89, "Dates"] = '2001-06-07'
dates.loc[90, "Dates"] = '2001-03-01'
dates.loc[91, "Dates"] = '2000-12-07'
dates.loc[92, "Dates"] = '2000-09-07'
dates.loc[93, "Dates"] = '2000-06-01'
dates.loc[94, "Dates"] = '2000-03-02'
dates.loc[95, "Dates"] = '1999-12-02'
dates.loc[96, "Dates"] = '1999-09-02'
dates.loc[97, "Dates"] = '1999-06-03'
dates.loc[98, "Dates"] = '1999-03-04'

#Converting to datetype and subtracting the three weeks
dates["Dates"] = pd.to_datetime(dates["Dates"])-datetime.timedelta(days=21)

In [313]:
path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\ECB projections\\"

spring = dates[dates['Dates'].dt.month == (5 or 6)].iloc[::-1]
spring.to_csv(path1+'spring.csv')

summer = dates[dates['Dates'].dt.month == (8 or 9)].iloc[::-1]
summer.to_csv(path1+'summer.csv')

winter = dates[dates['Dates'].dt.month == (2 or 3)].iloc[::-1]
winter.to_csv(path1+'winter.csv')

autumn = dates[dates['Dates'].dt.month == (11 or 12)].iloc[::-1]
autumn.to_csv(path1+'autumn.csv')

# From Here it is another part of the code, do not run the above again

Create find closest date here

In [12]:
#Loading and adjusting data
path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\ECB projections\\"

for var in ['RealGDP', 'HCIP']:

    # Print the resulting DataFrame
    csvfile_path = path1 + f"{var}.csv"
    csvfile=pd.read_csv(csvfile_path, header=0, skiprows = lambda x: x in range(6))
    csvfile['Dates']=pd.to_datetime(csvfile['Dates'], format="%d/%m/%Y")

    #Splitting column string name '2014-Q1' into '2014' and '1'
    listy = csvfile.columns[2:].str.split('-Q', expand=True)

    #Rebuilding names swiotiching back to 'Q1 2014' via list comprehension
    kappa = ['Q'+listy[a][1] + ' ' + listy[a][0] for a in range(len(csvfile.columns)-2) ]

    #Creating dictionary to rename the dataframe
    almir = dict(zip(csvfile.columns[2:], kappa))

    #Renaming the dataframe
    csvfile.rename(columns= almir , inplace = True)
    if var == 'RealGDP':
        rdp = csvfile

    elif var == 'HCIP':
        cip = csvfile

In [15]:
csvfile_path = path1 + "Unemp.csv"
csvfile=pd.read_csv(csvfile_path, header=0, skiprows = lambda x: x in range(6))
csvfile['Dates']=pd.to_datetime(csvfile['Dates'], format="%d/%m/%Y")

unemp = csvfile

In [14]:
#Building dataset to run regressions

# Read the reference dates from source1.xlsx
source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'
df_source = pd.read_excel(source_file, header=0, sheet_name=1)

# Get the reference dates from the first column
reference_dates = df_source.iloc[:, 0]

# Create a DataFrame with the reference dates
result_df = pd.DataFrame(data=reference_dates)

# Add the quarter column
result_df['Quarter'] = pd.PeriodIndex(result_df['date'], freq='Q')

# Read the comp.csv file and get the fourth column
#csv_file = 'comp.csv'
#df_csv = pd.read_csv(csv_file, header=None)
#fourth_column = df_csv.iloc[8, :]

# Add the fourth column to the result DataFrame
#result_df['Fourth Column'] = fourth_column

path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\ECB projections\\"
kk = 'p'

for j in range(-1,3):
    
    queue = f"Quarter_{j if j >= 0 else 'p'}"
    
    #Loop begins here
    #Taking the following n forecasts
    result_df[queue] = result_df['Quarter'].apply(lambda x: x + j)
    #result_df['Next Quarter2'] = result_df['Quarter'].apply(lambda x: x + 2)

    result_df[queue] = pd.PeriodIndex(result_df[queue]).strftime('Q%q %Y')

#     #Getting indexes where the quarters change
#     idlist = result_df.index[result_df['Quarter'].ne(result_df['Quarter'].shift().bfill())].tolist()
#     #Adding zero
#     idlist.insert(0,0)
    
    count=0
    for var in [rdp, cip]:
        
        if count == 0: name = 'RealGDP'
        if count == 1: name = 'HCIP'
        count+=1
        # Print the resulting DataFrame
#         csvfile_path = path1 + f"{var}.csv"
#         pd.read_csv(csvfile_path, header=0, skiprows = lambda x: x in range(6))
         #Selecting only the target quarter to be replaced in result_df

        auxres = result_df
        #aux1 = auxres['date']
        #aux1 = aux1.apply(find_closest_date)
        #result_df['Close'] = auxres['date'].apply(find_closest_date)
        
        def klug(x):
            y1 = x[0] #find closest date -> row a
            y2 = x[1]

            aux = pd.DataFrame(var['Dates'].apply(lambda x: (x - y1)))
            aux1 = aux[aux['Dates']<datetime.timedelta(days=0)]

            if aux1.empty:
                date_aux = math.nan
                var_aux = math.nan
                revision_aux = math.nan
            else:
                indx = aux1.idxmax()
                date_aux = var.loc[indx,'Dates'].squeeze()

                #The value of the forecast at the given horizon
                var_aux = float(var.loc[indx,y2].squeeze())


                #Cmputing second dataframe to get revisions
                aux2 = aux[aux['Dates']< aux.loc[indx, 'Dates'].squeeze()]

                if aux2.empty:
                    revision_aux = math.nan
                else:    
                    indx2 = aux2.idxmax() 
                    revision_aux = float(var.loc[indx2,y2].squeeze())

            return date_aux, var_aux, revision_aux 
        #Unpacking date and the forecast value
        auxi = pd.DataFrame(auxres[['date', queue]].apply(klug, axis=1).to_list(), index=auxres.index)

        #result_df.loc[auxres.index,f"Closest_date_{j if j >= 0 else 'p'}"] = auxi[0]
        result_df.loc[auxres.index,f"{name}_{j if j >= 0 else 'p'}"] = auxi[1]
        result_df.loc[auxres.index,f"{name}_Rev_{j if j >= 0 else 'p'}"] = auxi[1] - auxi[2]

In [16]:
#Now getting unemploment

#Adding the year column
result_df['Year'] = pd.PeriodIndex(result_df['date'], freq='Y')






kk = 'p'

for j in range(-1,3):
    
    queue = f"Year_{j if j >= 0 else 'p'}"
    
    #Loop begins here
    #Taking the following n forecasts
    result_df[queue] = result_df['Year'].apply(lambda x: x + j)
    #result_df['Next Quarter2'] = result_df['Quarter'].apply(lambda x: x + 2)


    #Getting indexes where the quarters change
    idlist = result_df.index[result_df['Quarter'].ne(result_df['Quarter'].shift().bfill())].tolist()
    #Adding zero
    idlist.insert(0,0)
    

    for var in [unemp]:
        
        name = 'Unemployment'
        # Print the resulting DataFrame
#         csvfile_path = path1 + f"{var}.csv"
#         pd.read_csv(csvfile_path, header=0, skiprows = lambda x: x in range(6))
         #Selecting only the target quarter to be replaced in result_df

        auxres = result_df
        #aux1 = auxres['date']
        #aux1 = aux1.apply(find_closest_date)
        #result_df['Close'] = auxres['date'].apply(find_closest_date)
        
        def klug(x):
            y1 = x[0] #find closest date -> row a
            y2 = str(x[1])

            aux = pd.DataFrame(var['Dates'].apply(lambda x: (x - y1)))
            aux1 = aux[aux['Dates']<datetime.timedelta(days=0)]

            if aux1.empty:
                date_aux = math.nan
                var_aux = math.nan
                revision_aux = math.nan
            else:
                indx = aux1.idxmax()
                date_aux = var.loc[indx,'Dates'].squeeze()

                #The value of the forecast at the given horizon
                var_aux = float(var.loc[indx,y2].squeeze())


                #Cmputing second dataframe to get revisions
                aux2 = aux[aux['Dates']< aux.loc[indx, 'Dates'].squeeze()]

                if aux2.empty:
                    revision_aux = math.nan
                else:    
                    indx2 = aux2.idxmax() 
                    revision_aux = float(var.loc[indx2,y2].squeeze())

            return date_aux, var_aux, revision_aux 
        #Unpacking date and the forecast value
        auxi = pd.DataFrame(auxres[['date', queue]].apply(klug, axis=1).to_list(), index=auxres.index)

        #result_df.loc[auxres.index,f"Closest_date_{j if j >= 0 else 'p'}"] = auxi[0]
        result_df.loc[auxres.index,f"{name}_{j if j >= 0 else 'p'}"] = auxi[1]
        result_df.loc[auxres.index,f"{name}_Rev_{j if j >= 0 else 'p'}"] = auxi[1] - auxi[2]

In [24]:
forecasts.iloc[8:,:].isnull().values.any()
#From 8th obs there are no nan's left (excluding +2 periods forecast for unemployment)

False

In [369]:
variables = ['HCIP', 'Unemployment', 'RealGDP']
und = ['_', '_Rev_']
index = [0,1,2,'p']
forecasts = result_df[[f"{v}{u}{i}" for v,u,i in itertools.product(variables, und, index) if not ((v=='Unemployment') and ((i in [2, 1, 'p']) and (u != '_Rev_')))]]
forecasts = forecasts.loc[:, forecasts.columns!='Unemployment_Rev_2']
forecasts.to_csv(path1+'forecasts.csv')

In [84]:
forecasts[[v for v in forecasts.columns if v not in ['HCIP_0', 'HCIP_1']]]

,HCIP_2,HCIP_p,HCIP_Rev_0,HCIP_Rev_1,HCIP_Rev_2,HCIP_Rev_p,Unemployment_0,Unemployment_Rev_0,Unemployment_Rev_1,Unemployment_Rev_p,RealGDP_0,RealGDP_1,RealGDP_2,RealGDP_p,RealGDP_Rev_0,RealGDP_Rev_1,RealGDP_Rev_2,RealGDP_Rev_p
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.9,0.9,NaN,NaN,NaN,NaN,10.6,NaN,NaN,NaN,0.5,0.5,0.5,0.5,NaN,NaN,NaN,NaN
3,0.9,0.9,NaN,NaN,NaN,NaN,10.6,NaN,NaN,NaN,0.5,0.5,0.5,0.5,NaN,NaN,NaN,NaN
4,0.9,0.9,NaN,NaN,NaN,NaN,10.6,NaN,NaN,NaN,0.5,0.5,0.5,0.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,4.0,5.6,1.9,2.2,2.1,1.5,7.3,0.0,0.3,0.0,1.0,1.0,0.8,0.2,-0.4,-0.2,0.1,-0.2
287,6.4,6.1,1.9,2.1,2.4,0.5,6.8,-0.5,-0.4,0.0,0.2,0.4,0.5,0.3,-0.8,-0.6,-0.3,0.1
288,4.7,7.5,2.1,2.4,2.2,1.9,6.8,-0.5,-0.4,0.0,0.4,0.5,0.6,0.2,-0.6,-0.3,-0.1,-0.8
289,8.0,8.0,1.8,2.8,3.3,0.5,6.7,-0.1,0.1,0.0,0.1,-0.1,0.0,0.7,-0.3,-0.6,-0.6,0.5


# Computing instruments

In [25]:
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg



In [26]:
#Loading dataframes
source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'
df_source = pd.read_excel(source_file, header=0, sheet_name=1)

path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\ECB projections\\"
forecasts = pd.read_csv((path1+'forecasts.csv'), header=0, index_col=0)
#forecasts = result_df

# Get the reference dates from the first column
reference_dates = df_source.iloc[:, 0]

# Create a DataFrame with the reference dates
result_df = pd.DataFrame(data=reference_dates)

In [22]:
variables = ['HCIP', 'Unemployment', 'RealGDP']
und = ['_', '_Rev_']
index = [0,1,2,'p']
forecasts = result_df[[f"{v}{u}{i}" for v,u,i in itertools.product(variables, und, index) if not ((v=='Unemployment') and ((i in [2, 1, 'p']) and (u != '_Rev_')))]]

forecasts = forecasts.loc[:, forecasts.columns!='Unemployment_Rev_2']
#forecasts.to_csv(path1+'forecasts.csv')

In [27]:
forecasts.iloc[1,:]

HCIP_0               NaN
HCIP_1               NaN
HCIP_2               NaN
HCIP_p               NaN
HCIP_Rev_0           NaN
HCIP_Rev_1           NaN
HCIP_Rev_2           NaN
HCIP_Rev_p           NaN
Unemployment_0       NaN
Unemployment_Rev_0   NaN
Unemployment_Rev_1   NaN
Unemployment_Rev_p   NaN
RealGDP_0            NaN
RealGDP_1            NaN
RealGDP_2            NaN
RealGDP_p            NaN
RealGDP_Rev_0        NaN
RealGDP_Rev_1        NaN
RealGDP_Rev_2        NaN
RealGDP_Rev_p        NaN
Name: 1, dtype: float64

In [28]:
def sum_values_by_month(df):
    # Convert the month column to datetime type
    #df['Month'] = pd.to_datetime(df['Month'])
    
    #df = df.groupby('Month')['Values'].sum().reset_index()

    
    # Create a new dataframe with a complete range of months
    min_month = df['Month'].min().to_timestamp()
    max_month = df['Month'].max().to_timestamp() + datetime.timedelta(days=32)
    complete_months = pd.date_range(start=min_month, end=max_month, freq='M')
    complete_df = pd.DataFrame({'Month': complete_months})
    
    complete_df['Month'] = pd.PeriodIndex(complete_df['Month'], freq='M')
    # Merge the original dataframe with the complete dataframe to fill missing months with NaN values
    merged_df = pd.merge(complete_df, df, on='Month', how='left')

    # Group the merged dataframe by month and calculate the sum of values for each month
    #result = merged_df.groupby('Month')['Values'].sum().reset_index()
    
    result = merged_df
    # Fill NaN values with 0
    result['Values'] = result['Values'].fillna(0)

    return result

In [29]:
#Loop version
source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'

path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\ECB projections\\Instruments\\"
df_source = pd.read_excel(source_file, header=0, sheet_name=1)


#Period a = initial month/year index, b final month/year index
a = 71
b = 268 # Nocovid

for k in ['SW', '1M', '3M', '6M', '1Y', '2Y']:

    y = df_source[f"OIS_{k}"][a:b]
    x = forecasts.iloc[a:b,:]
    #add constant to predictor variables
    x = sm.add_constant(x)

    #fit linear regression model
    model = sm.OLS(y, x, missing='drop').fit()


    #path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\"
    instrument = model.resid
    # Get the range of indexes to include
    #index_range = range(instrument.index.min(), instrument.index.max() + 1)


    ##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
    # Reindex the DataFrame with the full index range
    #instrument = instrument.reindex(index=range(0,291))

    instrument.to_csv(path3+f"instrument{k}_autcor.csv")
    
    resid = pd.DataFrame(instrument, columns=['Values'])
    #Adding month mark
    result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')
    
    #Adding the month column to the new dataframe
    resid['Month'] = result_df.loc[resid['Values'].index, 'Month']
    
    MPI = resid.groupby('Month')['Values'].sum().reset_index()
    
    atreg12 = AutoReg(MPI['Values'], lags=12).fit()

    realinstrument =  pd.DataFrame(atreg12.resid, columns=['Values'])
    realinstrument['Month'] = MPI.loc[12:, 'Month']
    realinstrument = sum_values_by_month(realinstrument)
    realinstrument.to_csv(path3+f"instrument{k}.csv")

In [8]:
math.log(10)

2.302585092994046

# Pure instrument

In [121]:
#Pure instrument

source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'
df_source = pd.read_excel(source_file, header=0, sheet_name=1)

# Get the reference dates from the first column
reference_dates = df_source.iloc[:, 0]

# Create a DataFrame with the reference dates
result_df = pd.DataFrame(data=reference_dates)

#Loop version

#Period a = initial month/year index, b final month/year index
a = 71
b = 268

for k in ['SW', '1M', '3M', '6M', '1Y', '2Y']:

    y = df_source[f"OIS_{k}"][a:b]
    #x = forecasts.iloc[a:,:]
    #add constant to predictor variables
    #x = sm.add_constant(x)

    #fit linear regression model
    #model = sm.OLS(y, x, missing='drop').fit()


    path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Pure Instruments\\"
    instrument = y
    # Get the range of indexes to include
    #index_range = range(instrument.index.min(), instrument.index.max() + 1)


    ##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
    # Reindex the DataFrame with the full index range
    #instrument = instrument.reindex(index=range(0,291))

    #instrument.to_csv(path3+f"instrument{k}.csv")
    
    resid = pd.DataFrame(instrument).rename(columns={f"OIS_{k}":'Values'})
    #Adding month mark
    result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')
    
    #Adding the month column to the new dataframe
    resid['Month'] = result_df.loc[resid['Values'].index, 'Month']
    
    MPI = resid.groupby('Month')['Values'].sum().reset_index()
    
    #atreg12 = AutoReg(MPI['Values'], lags=12, missing='drop').fit()

    #realinstrument =  pd.DataFrame(atreg12.resid, columns=['Values'])
    #realinstrument['Month'] = MPI.loc[12:, 'Month']
    realinstrument = sum_values_by_month(MPI)
    realinstrument.to_csv(path3+f"realinstrument{k}.csv")

In [31]:
forecasts.iloc[1,:]

HCIP_0               NaN
HCIP_1               NaN
HCIP_2               NaN
HCIP_p               NaN
HCIP_Rev_0           NaN
HCIP_Rev_1           NaN
HCIP_Rev_2           NaN
HCIP_Rev_p           NaN
Unemployment_0       NaN
Unemployment_Rev_0   NaN
Unemployment_Rev_1   NaN
Unemployment_Rev_p   NaN
RealGDP_0            NaN
RealGDP_1            NaN
RealGDP_2            NaN
RealGDP_p            NaN
RealGDP_Rev_0        NaN
RealGDP_Rev_1        NaN
RealGDP_Rev_2        NaN
RealGDP_Rev_p        NaN
Name: 1, dtype: float64

In [39]:
#df_source = df_source.set_index(pd.Series(range(0,291)))
#forecasts = forecasts.set_index(pd.Series(range(0,291)))
df_source = pd.read_excel(source_file, header=0, sheet_name=3)

y = df_source['OIS_SW'][71:268]
#x1 = forecasts[[v for v in forecasts.columns if v not in ['Unemployment_Rev_1', 'RealGDP_Rev_2', 'RealGDP_Rev_1','RealGDP_Rev_p', 'RealGDP_p','RealGDP_1','RealGDP_2','HCIP_Rev_p','HCIP_Rev_1', 'HCIP_Rev_2','HCIP_p', 'HCIP_1','HCIP_2']]]
#x1 = forecasts[[v for v in forecasts.columns if v not in ['Unemployment_Rev_1','Unemployment_1','Unemployment_2','Unemployment_p', 'Unemployment_Rev_2','HCIP_Rev_2','HCIP_Rev_1','RealGDP_Rev_1','HCIP_0','HCIP_p','RealGDP_2','RealGDP_p','HCIP_Rev_0','Unemployment_0','RealGDP_1','Unemployment_Rev_0','HCIP_Rev_p','RealGDP_Rev_2', 'RealGDP_Rev_0', 'RealGDP_Rev_p', 'RealGDP_0', 'HCIP_1', 'HCIP_2']]]

#x = x1.iloc[71:268,:]


x = forecasts.iloc[71:268,:]
#add constant to predictor variables
#x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x, missing='drop').fit()

#view model summary
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 OIS_SW   R-squared (uncentered):                   0.082
Model:                            OLS   Adj. R-squared (uncentered):             -0.022
Method:                 Least Squares   F-statistic:                             0.7894
Date:                Fri, 23 Jun 2023   Prob (F-statistic):                       0.724
Time:                        21:30:43   Log-Likelihood:                         -472.34
No. Observations:                 197   AIC:                                      984.7
Df Residuals:                     177   BIC:                                      1050.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [219]:
from statsmodels.stats.diagnostic import het_white
#perform White's test
white_test = het_white(model.resid,  model.model.exog)

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

#print results of White's test
print(dict(zip(labels, white_test)))

{'Test Statistic': 22.678319732495826, 'Test Statistic p-value': 0.06570956172257252, 'F-Statistic': 1.704884174384756, 'F-Test p-value': 0.059670152205849196}


In [72]:
min_month = realinstrument['Month'].min().to_timestamp()
max_month = realinstrument['Month'].max().to_timestamp() 
complete_months = pd.date_range(start=min_month, end=max_month, freq='M')
complete_df = pd.DataFrame({'Month': complete_months})
complete_months

DatetimeIndex(['2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30',
               '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31',
               '2003-09-30', '2003-10-31',
               ...
               '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
               '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31',
               '2022-09-30', '2022-10-31'],
              dtype='datetime64[ns]', length=238, freq='M')

In [60]:
max_month

Timestamp('2019-12-01 00:00:00')

In [64]:
help(datetime.timedelta)

Help on class timedelta in module datetime:

class timedelta(builtins.object)
 |  Difference between two datetime values.
 |  
 |  timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
 |  
 |  All arguments are optional and default to 0.
 |  Arguments may be integers or floats, and may be positive or negative.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  